# Consigna

Link a la consigna: https://piazza.com/class_profile/get_resource/jkr2voxi1yw4wt/jnj49wdk7mfw9

Tenemos  información  sobre recetas en 3 RDD de Spark. 

Recetas: (id_receta, nombre, tiempo_preparación, dificultad) 

Ingredientes: (id_ingrediente, nombre) 

Ingredientes   por   Receta:  (id_receta, id_ingrediente, cantidad) 

Se pide: 

a)  Obtener  el  nombre  de  todas  las recetas que tengan Cordero. (7 puntos) 

b)  Calcular  la  cantidad  total  de  cada ingrediente si queremos hacer todas las recetas con Cordero que sean fáciles. (8 puntos)

In [ ]:
import pyspark
sc = pyspark.SparkContext.getOrCreate()
sqlContext = pyspark.sql.SQLContext(sc)

In [ ]:
rdd_recetas = sqlContext.read.csv('../data/2018C2_recetas.csv').rdd
rdd_ingredientes = sqlContext.read.csv('../data/2018C2_ingredientes.csv').rdd
rdd_ingredientesxreceta = sqlContext.read.csv('../data/2018C2_ingredientesporreceta.csv').rdd

## Punto A

Tener en cuenta que **NO esta bien hacer los collects intermedios**. Solo el final. Los hago para visualizar mejor los pasos del ejercicio, pero el collect es una acción costosa, solo debe ser utilizada cuando se tiene el absoluto mínimo necesario.

Filtro ingredientes para tener solo 'Cordero' como un RDD y en un inner join poder filtrar las recetas

In [ ]:
cordero = rdd_ingredientes.filter(lambda x:x[1] == 'Cordero')
cordero.collect()

Me quedo con la tupla (id_ingrediente, id_receta) para despues hacer un join y solo quedarme con las recetas con cordero. Depsués, me quedo con los id de recetas

In [ ]:
recetasconcordero = rdd_ingredientesxreceta.map(lambda x:(x[1],x[0]))\
                                           .join(cordero)\
                                           .map(lambda x:(x[1][0],None))
recetasconcordero.collect()

De las recetas solo me quedo con su id y nombre. Luego, otra vez haciendo un inner join como filtro, me quedo con las recetas de corderos. De ahi, me quedo con los nombres.

Finalizo con un collect.

In [ ]:
nombresrecetacordero = rdd_recetas.map(lambda x:(x[0],x[1]))\
                                  .join(recetasconcordero)\
                                  .map(lambda x:(x[1][0]))\

nombresrecetacordero.collect()

## Punto B

Me quedo con las recetas fáciles.

In [ ]:
recetasfaciles = rdd_recetas.filter(lambda x:x[3]=='facil')\
                            .map(lambda x:(x[0],None))

recetasfaciles.collect()

Inner join me sirve de filtro --> Tengo recetas faciles con cordero

In [ ]:
# Copiando del ejercicio anterior la variable recetas con cordero:
recetasconcordero = rdd_ingredientesxreceta.map(lambda x:(x[1],x[0]))\
                                           .join(cordero)\
                                           .map(lambda x:(x[1][0],None))

In [ ]:
recetasfacilesconcordero = recetasfaciles.join(recetasconcordero)\
                                         .map(lambda x:(x[0],None))

recetasfacilesconcordero.collect()

Convierto el RDD de ingredientes x receta para que sea de tipo clave, valor, para poder hacer un join normalmente

In [ ]:
rdd_ingredientesxreceta_mod = rdd_ingredientesxreceta.map(lambda x:(x[0],(x[1],x[2])))
rdd_ingredientesxreceta_mod.collect()

Me quedo con la tupla (id_ingrediente, cantidad) y sumo las cantidades por ingrediente)

In [ ]:
ingredientesrecetasfacilesconcordero = rdd_ingredientesxreceta_mod.join(recetasfacilesconcordero)\
                                                              .map(lambda x:(x[1][0][0],x[1][0][1]))\
                                                              .reduceByKey(lambda x,y:x+y)

ingredientesrecetasfacilesconcordero.collect()

Me quedo con la tupla (nombre_ingrediente, cantidad)

In [ ]:
ingredientesrecetasfacilesconcordero_connombre = ingredientesrecetasfacilesconcordero.join(rdd_ingredientes)\
                                                              .map(lambda x:(x[1][1],x[1][0]))

ingredientesrecetasfacilesconcordero_connombre.collect()